In [87]:
%pip install keras-tuner tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [88]:
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import pandas as pd

In [89]:
dataset_path = 'modified_star_classification.csv'

In [90]:
df = pd.read_csv(dataset_path)

In [91]:
df.shape

(99999, 7)

In [92]:
df.head()

,uv,green,red,near_infrared,infrared,redshift,class
0,23.87882,22.27530,20.39501,19.16573,18.79371,0.634794,0
1,24.77759,22.83188,22.58444,21.16812,21.61427,0.779136,0
2,25.26307,22.66389,20.60976,19.34857,18.94827,0.644195,0
3,22.13682,23.77656,21.61162,20.50454,19.25010,0.932346,0
4,19.43718,17.58028,16.49747,15.97711,15.54461,0.116123,0


In [93]:
X = df.drop('class', axis=1)
y = df['class']

In [94]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=42)

In [95]:
X_train.dtypes

uv               float64
green            float64
red              float64
near_infrared    float64
infrared         float64
redshift         float64
dtype: object

In [96]:
X_val.dtypes

uv               float64
green            float64
red              float64
near_infrared    float64
infrared         float64
redshift         float64
dtype: object

In [97]:
y_train.dtypes

dtype('int64')

In [98]:
y_val.dtypes

dtype('int64')

In [99]:
shape = X_train.shape[1]
shape

6

In [100]:
def build_model(hp):
  shape = X_train.shape[1]
  input_layer = keras.Input(shape=(shape,))

  x = layers.Dense(
      hp.Choice("units_1", [32, 64, 128]),
      activation=hp.Choice("activation", ["relu", "tanh"])
  )(input_layer)

  if hp.Boolean("second_layer"):
    x = layers.Dense(
        hp.Choice("units_2", [32, 64]),
        activation=hp.Choice("activation_2", ["relu", "tanh"])
    )(x)

  output_layer = layers.Dense(3, activation="softmax")(x)

  model = keras.Model(inputs=input_layer, outputs=output_layer)

  model.compile(
      optimizer=keras.optimizers.Adam(
          learning_rate=hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])
      ),
      loss="sparse_categorical_crossentropy",
      metrics=["accuracy"]
  )

  return model

In [101]:
tuner = kt.GridSearch(
    build_model,
    objective="val_accuracy",
    max_trials=10,
    directory="tuner1_dir",
    project_name="star_classification"
)

In [102]:
tuner.search(X_train, y_train, epochs=50, validation_data=(X_val, y_val), verbose=1)

Trial 10 Complete [00h 01m 08s]
val_accuracy: 0.9701499938964844

Best val_accuracy So Far: 0.9701499938964844
Total elapsed time: 00h 11m 17s


In [103]:
best_hp = tuner.get_best_hyperparameters(1)[0]

In [104]:
print(best_hp)

In [105]:
print("Melhores hiperparâmetros: ")
for name in best_hp.values:
  print(f"{name}: {best_hp.get(name)}")

Melhores hiperparâmetros: 
units_1: 32
activation: relu
second_layer: True
learning_rate: 0.001
units_2: 64
activation_2: relu
